In [28]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Step 2 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9044/26545/118033/'>link</a>
<br>Сам запрос к API должен состоять из нескольких параметров. Пример обращения к поиску (Search):

r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=QUERY')
res = r.json()

где QUERY – ключевое слово.

Предположим, ваша задача – найти все произведения, связанные с подсолнухами. Выполните поиск по слову "sunflowers". Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.

В качестве ответа выберите id объектов (3), которые присутствуют в полученном списке: [16822570, 2032, 437329, 436524, 20149]

In [69]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()

In [70]:
sunflower_ids = res['objectIDs']
len(sunflower_ids)

85

In [9]:
ids = [16822570, 2032, 437329, 436524, 20149]

for i in ids:
    print(f'{i} is in {i in sunflower_ids}')

16822570 is in False
2032 is in True
437329 is in True
436524 is in True
20149 is in False


Step 3 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9044/26545/118034/'>link</a>
<br>Отлично! На предыдущем шаге мы получили результаты поиска. Следущий этап – собрать информацию об этих объектах.

Пример текста запроса для получения информации об одном объекте (Object):

r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/OBJECT_ID')
<br>obj_test = r_test.json()

где OBJECT_ID – id объекта.

Выполните запрос для объекта 437980. Для проверки соотнесите полученные параметры с их значениями:

In [11]:
obj_id = 437980
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{obj_id}')
obj_test = r_test.json()

In [17]:
print('department = {}'.format(obj_test['department']))
print('objectName = {}'.format(obj_test['objectName']))
print('title = {}'.format(obj_test['title']))
print('culture = {}'.format(obj_test['culture']))
print('artistDisplayName = {}'.format(obj_test['artistDisplayName']))
print('objectID = {}'.format(obj_test['objectID']))
print('GalleryNumber = {}'.format(obj_test['GalleryNumber']))


department = European Paintings
objectName = Painting
title = Cypresses
culture = 
artistDisplayName = Vincent van Gogh
objectID = 437980
GalleryNumber = 825


Step 4 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9044/26545/118035/'>link</a>
<br>Теперь напишите цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге. Результаты запишите в датафрейм, а именно – следующие параметры:

objectID – id объекта
<br>title – название
<br>artistDisplayName – автор
<br>department – департамент
<br>objectBeginDate – дата (начало)
<br>objectEndDate – дата (конец)
<br>period – название периода
<br>objectName – название/категория объекта
<br>culture – культура

Сколько объектов относятся к японской культуре? (culture –Japan)

In [21]:
props = ['objectID', 'title', 'artistDisplayName', 'department', 'objectBeginDate', 'objectEndDate', 'period', 'objectName', 'culture']

In [46]:
data = []
errors = []
for obj_id in sunflower_ids:
    resp = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{obj_id}')
    if resp.status_code == 200:
        obj = resp.json()
        row = [obj[p] for p in props]
        data.append(row)
    else:
        errors.append({'obj_id':obj_id, 'status_code':resp.status_code})

obj_df = pd.DataFrame(data, columns=props)

In [48]:
errors

[{839296: 404}, {704667: 404}, {696949: 404}]

In [49]:
obj_df.isna().sum()

objectID             0
title                0
artistDisplayName    0
department           0
objectBeginDate      0
objectEndDate        0
period               0
objectName           0
culture              0
dtype: int64

In [52]:
obj_df.groupby(by='culture') \
    .agg({'objectID':'count'})

,objectID
culture,
,71
American,8
Ancestral Pueblo,1
British,1
Japan,1


Step 5 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9044/26545/118036/'>link</a>
<br>Теперь попробуем построить более сложный запрос с помощью секции "Search"! На этот раз параметры необходимо передать через "&", например, departmentId (id департамента – 1) и q (ключевое слово – cat):

r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=1&q=cat')
<br>cats = r.json()

Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. <br>Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.

Note: выполнение цикла может занять некоторое время (~5 минут), поэтому проверять его работоспособность лучше на более маленькой выборке!

В качестве ответа укажите, в каком периоде (period) было выполнено наибольшее количество работ, представленных в музее?

In [131]:
resp = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = resp.json()

In [132]:
cat_ids = cats['objectIDs']
len(cat_ids)

2909

In [134]:
max_count = 1000
cat_ids = cat_ids[0:min(len(cat_ids), max_count)]
len(cat_ids)

1000

In [135]:
data = []
errors = []
for obj_id in cat_ids:
    resp = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{obj_id}')
    if resp.status_code == 200:
        obj = resp.json()
        row = [obj[p] for p in props]
        data.append(row)
    else:
        errors.append({'obj_id':obj_id, 'status_code':resp.status_code})

cats_df = pd.DataFrame(data, columns=props)

In [136]:
errors

[]

In [143]:
cats_df.to_csv('cats.csv', index=False)

In [138]:
cats_df.shape

(1000, 9)

In [142]:
cats_df.groupby('period', as_index=False) \
    .agg({'objectID':'count'}) \
    .rename(columns={'objectID':'counts'}) \
    .sort_values('counts', ascending=False) \
    .head()

,period,counts
16,Edo period (1615–1868),455
0,,95
75,Qing dynasty (1644–1911),49
43,Ming dynasty (1368–1644),49
63,Muromachi period (1392–1573),30
